In [1]:
from qiskit import QuantumCircuit, transpile, Aer, execute
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np
from qiskit.quantum_info import DensityMatrix
import torch
import pickle  # For exporting the variables
from tqdm import tqdm


In [2]:
def initialize_all_zeros(nr_qubits):
    # Create a quantum circuit with N qubits
    qc = QuantumCircuit(nr_qubits)

    # Initialize all qubits in the |0⟩ state
    for qubit in range(nr_qubits):
        qc.initialize([1, 0], qubit)

    return qc

In [3]:
def apply_cnot_chain(qc, control_qubit, nr_qubits):
    # Apply CNOT gates from the control qubit to the list of target qubits
    for target_qubit in range(nr_qubits-1):
        qc.cx(control_qubit, target_qubit+1)

# def random_measurement_basis(nr_qubits): #applies to 1 qubit
#     measurement_basis = [1, 2, 3]
#     return [random.choice(measurement_basis) for _ in range(nr_qubits)]


def measurement_bases_N(nr_qubits):
    # Generate nr_qubits random numbers from the set {x=1, y=2, z=3}
    # random_bases = np.random.choice([1, 2, 3], size=nr_qubits) # X=1 Y=2 Z=3

    random_bases = np.random.choice([1, 2, 3], size=nr_qubits)

    return random_bases

In [4]:
if __name__ == "__main__":
    nr_qubits = 7
    n_sample = 200  # amount of times the shadows are generated and measured with 1 shot each

    # Initialize a list to store the rows
    rows = []

    # Initialize an empty list to store the measured result and bases in the correct format for the AI model
    measurement_results_in_specific_format = np.zeros(
        (n_sample, nr_qubits), dtype=int)
    measurement_bases_in_specific_format = []

    simulator = Aer.get_backend('aer_simulator')

    for i in tqdm(range(n_sample)):
        # Create a quantum circuit with N qubits
        qc = initialize_all_zeros(nr_qubits)

        # Usually just the first qubit. Hardcoded because we don't need the control
        control_qubit = 0

        # Apply a Hadamard gate to qubit 0
        qc.h(control_qubit)

        apply_cnot_chain(qc, control_qubit, nr_qubits)

        # Choose random measurement basis for each qubit
        measurement_bases = measurement_bases_N(nr_qubits)

        # measurement_bases = np.array([1, 1, 1, 3]) #To check a specific case
        # print('measurement_bases', measurement_bases)

        # Store the measurement bases in the list
        measurement_bases_in_specific_format.append(measurement_bases)

        # Apply the measurement bases to the qubits
        for qubit in range(nr_qubits):
            if measurement_bases[qubit] == 1:
                qc.h(qubit)
            elif measurement_bases[qubit] == 2:
                qc.sdg(qubit)
                qc.h(qubit)

        # Add measurements for all qubits in the Z basis
        qc.measure_all()


100%|███████████████████████████████████████| 200/200 [00:00<00:00, 1205.06it/s]


In [11]:
job = execute(qc, simulator, shots=1)
result = job.result()
counts = result.get_counts(qc)

        # Iterate through the qubits and add their measurement results to the list
for qubit in range(nr_qubits):
    basis = measurement_bases[qubit]
            # Extract the result
    result = int(list(counts.keys())[0][nr_qubits - 1 - qubit])
    rows.append(pd.DataFrame(
                {"Measurement Basis": [basis], "Measured Result": [result]}))

    measurement_results_in_specific_format[i][qubit] = result

        # print(measurement_results_in_specific_format[_]) #To compare them to the ones in the paper

        # Concatenate the rows into the DataFrame and reset the index
    df = pd.concat(rows, ignore_index=True)

        # Display the DataFrame
        # print('df',df)

obs_before_tensor = measurement_bases_in_specific_format
    # print('obs_before_tensor', obs_before_tensor)

out_before_tensor = [np.array(row) for row in measurement_results_in_specific_format]
    # print('out_before_tensor', out_before_tensor)
    # Create a dictionary to store your variables
variables_to_export = {
    'obs_before_tensor': obs_before_tensor,
    'out_before_tensor': out_before_tensor
    }
    
path = f'data/exported_qubits_{nr_qubits}_samples_{n_sample}.pkl'

try:
    with open(path, 'rb') as file:
        loaded_variables = pickle.load(file)
except FileNotFoundError:
    loaded_variables = {
    'obs_before_tensor': [],
    'out_before_tensor': []
}
except Exception:
    loaded_variables = None


if loaded_variables is not None:
# Export the dictionary to a file
    with open(path, 'wb') as file:


        loaded_variables['obs_before_tensor'] += obs_before_tensor
        loaded_variables['out_before_tensor'] += out_before_tensor
        print(len(loaded_variables['obs_before_tensor']))
        pickle.dump(loaded_variables, file)


200
